### CNN on CIFR Assignment:

1.  Please visit this link to access the state-of-art DenseNet code for reference - DenseNet - cifar10 notebook link
2.  You need to create a copy of this and "retrain" this model to achieve 90+ test accuracy. 
3.  You cannot use DropOut layers.
4.  You MUST use Image Augmentation Techniques.
5.  You cannot use an already trained model as a beginning points, you have to initilize as your own
6.  You cannot run the program for more than 300 Epochs, and it should be clear from your log, that you have only used 300 Epochs
7.  You cannot use test images for training the model.
8.  You cannot change the general architecture of DenseNet (which means you must use Dense Block, Transition and Output blocks as mentioned in the code)
9.  You are free to change Convolution types (e.g. from 3x3 normal convolution to Depthwise Separable, etc)
10. You cannot have more than 1 Million parameters in total
11. You are free to move the code from Keras to Tensorflow, Pytorch, MXNET etc. 
12. You can use any optimization algorithm you need. 
13. You can checkpoint your model and retrain the model from that checkpoint so that no need of training the model from first if you lost at any epoch while training. You can directly load that model and Train from that epoch. 

In [1]:
#https://towardsdatascience.com/review-densenet-image-classification-b6631a8ef803
#We can read from the paper itself that the team got a test error of 3.64% on CIFAR10 Dataset using a 40 layer model with k=12.
#The above article tells us how to check if the k value is 12.
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten,ReLU, concatenate,Conv2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
#https://www.kaggle.com/code/thetenaciousguy/cnn-densenet-first3x3-l18
#https://towardsdatascience.com/densenet-on-cifar10-d5651294a1a8

In [3]:
%load_ext tensorboard


In [6]:
# Load CIFAR10 Data
num_classes=10
(X_train_1, y_train_1), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_cv, y_train, y_cv=train_test_split(X_train_1, y_train_1, test_size=0.2, stratify=y_train_1)

X_test=X_test/255
x_cv=x_cv/255
# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 
y_cv = tf.keras.utils.to_categorical(y_cv, num_classes) 

In [4]:
'''import pickle

with open ('pickle_files/x_train.pkl', 'wb') as f:
    f= pickle.dump(x_train, f)
    
with open ('pickle_files/y_train.pkl', 'wb') as f:
    f= pickle.dump(y_train, f)

with open ('pickle_files/x_test.pkl', 'wb') as f:
    f= pickle.dump(X_test, f)

with open ('pickle_files/y_test.pkl', 'wb') as f:
    f= pickle.dump(y_test, f)
    
with open ('pickle_files/x_cv.pkl', 'wb') as f:
    f= pickle.dump(x_cv, f)
    
with open ('pickle_files/y_cv.pkl', 'wb') as f:
    f= pickle.dump(y_cv, f)'''

In [3]:
import pickle

with open ('pickle_files/x_train.pkl', 'rb') as f:
    x_train= pickle.load(f)
    
with open ('pickle_files/y_train.pkl', 'rb') as f:
    y_train= pickle.load( f)

with open ('pickle_files/x_test.pkl', 'rb') as f:
    X_test= pickle.load(f)

with open ('pickle_files/y_test.pkl', 'rb') as f:
    y_test= pickle.load(f)
    
with open ('pickle_files/x_cv.pkl', 'rb') as f:
    x_cv= pickle.load(f)
    
with open ('pickle_files/y_cv.pkl', 'rb') as f:
    y_cv= pickle.load(f)
    
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


In [4]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    zca_epsilon=1e-06,
        rotation_range=15,  #  rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # zoom image 
        width_shift_range=0.20,  #  shift width
        height_shift_range=0.20,  # shift height
        horizontal_flip=True, rescale=1./255,fill_mode='nearest')  # flip images



train_datagen.fit(x_train)

In [5]:
img_height, img_width, channel

(32, 32, 3)

In [6]:
# Dense Block
#In a DenseNet we have a dense block where we have Batch Norm>Relu>Conv>DropOut and all the features are concatenated at each convolution.

def denseblock(input, num_filter, dropout_rate):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter, dropout_rate):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [7]:
total_layers=40
l = (total_layers-4)/3
l

12.0

In [8]:
# Hyperparameters
batch_size = 128 #As per the research Paper
num_classes = 10
epochs = 300
total_layers=40
l = 18
num_filter = 14
compression = 0.9
dropout_rate = 0

In [11]:
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (3,3), use_bias=False ,padding='same',kernel_initializer='he_uniform',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4))(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model=Model(inputs=input, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 14)   378         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 14)  56          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 14)   0           ['batch_normalization[0][0]']

                                                                                                  
 concatenate_7 (Concatenate)    (None, 32, 32, 110)  0           ['concatenate_6[0][0]',          
                                                                  'conv2d_8[0][0]']               
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 32, 32, 110)  440        ['concatenate_7[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 activation_8 (Activation)      (None, 32, 32, 110)  0           ['batch_normalization_8[0][0]']  
                                                                                                  
 conv2d_9 (Conv2D)              (None, 32, 32, 12)   11880       ['activation_8[0][0]']           
          

                                                                                                  
 batch_normalization_16 (BatchN  (None, 32, 32, 206)  824        ['concatenate_15[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_16 (Activation)     (None, 32, 32, 206)  0           ['batch_normalization_16[0][0]'] 
                                                                                                  
 conv2d_17 (Conv2D)             (None, 32, 32, 12)   22248       ['activation_16[0][0]']          
                                                                                                  
 concatenate_16 (Concatenate)   (None, 32, 32, 218)  0           ['concatenate_15[0][0]',         
                                                                  'conv2d_17[0][0]']              
          

                                                                                                  
 activation_24 (Activation)     (None, 16, 16, 72)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 16, 16, 12)   7776        ['activation_24[0][0]']          
                                                                                                  
 concatenate_23 (Concatenate)   (None, 16, 16, 84)   0           ['concatenate_22[0][0]',         
                                                                  'conv2d_25[0][0]']              
                                                                                                  
 batch_normalization_25 (BatchN  (None, 16, 16, 84)  336         ['concatenate_23[0][0]']         
 ormalization)                                                                                    
          

 conv2d_33 (Conv2D)             (None, 16, 16, 12)   18144       ['activation_32[0][0]']          
                                                                                                  
 concatenate_31 (Concatenate)   (None, 16, 16, 180)  0           ['concatenate_30[0][0]',         
                                                                  'conv2d_33[0][0]']              
                                                                                                  
 batch_normalization_33 (BatchN  (None, 16, 16, 180)  720        ['concatenate_31[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_33 (Activation)     (None, 16, 16, 180)  0           ['batch_normalization_33[0][0]'] 
                                                                                                  
 conv2d_34

                                                                  'conv2d_41[0][0]']              
                                                                                                  
 batch_normalization_41 (BatchN  (None, 8, 8, 48)    192         ['concatenate_38[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_41 (Activation)     (None, 8, 8, 48)     0           ['batch_normalization_41[0][0]'] 
                                                                                                  
 conv2d_42 (Conv2D)             (None, 8, 8, 12)     5184        ['activation_41[0][0]']          
                                                                                                  
 concatenate_39 (Concatenate)   (None, 8, 8, 60)     0           ['concatenate_38[0][0]',         
          

 ormalization)                                                                                    
                                                                                                  
 activation_49 (Activation)     (None, 8, 8, 144)    0           ['batch_normalization_49[0][0]'] 
                                                                                                  
 conv2d_50 (Conv2D)             (None, 8, 8, 12)     15552       ['activation_49[0][0]']          
                                                                                                  
 concatenate_47 (Concatenate)   (None, 8, 8, 156)    0           ['concatenate_46[0][0]',         
                                                                  'conv2d_50[0][0]']              
                                                                                                  
 batch_normalization_50 (BatchN  (None, 8, 8, 156)   624         ['concatenate_47[0][0]']         
 ormalizat

                                                                                                  
 conv2d_58 (Conv2D)             (None, 4, 4, 12)     1296        ['activation_57[0][0]']          
                                                                                                  
 concatenate_54 (Concatenate)   (None, 4, 4, 24)     0           ['average_pooling2d_2[0][0]',    
                                                                  'conv2d_58[0][0]']              
                                                                                                  
 batch_normalization_58 (BatchN  (None, 4, 4, 24)    96          ['concatenate_54[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_58 (Activation)     (None, 4, 4, 24)     0           ['batch_normalization_58[0][0]'] 
          

 concatenate_62 (Concatenate)   (None, 4, 4, 120)    0           ['concatenate_61[0][0]',         
                                                                  'conv2d_66[0][0]']              
                                                                                                  
 batch_normalization_66 (BatchN  (None, 4, 4, 120)   480         ['concatenate_62[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_66 (Activation)     (None, 4, 4, 120)    0           ['batch_normalization_66[0][0]'] 
                                                                                                  
 conv2d_67 (Conv2D)             (None, 4, 4, 12)     12960       ['activation_66[0][0]']          
                                                                                                  
 concatena

 batch_normalization_74 (BatchN  (None, 4, 4, 216)   864         ['concatenate_70[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_74 (Activation)     (None, 4, 4, 216)    0           ['batch_normalization_74[0][0]'] 
                                                                                                  
 conv2d_75 (Conv2D)             (None, 4, 4, 12)     23328       ['activation_74[0][0]']          
                                                                                                  
 concatenate_71 (Concatenate)   (None, 4, 4, 228)    0           ['concatenate_70[0][0]',         
                                                                  'conv2d_75[0][0]']              
                                                                                                  
 batch_nor

In [9]:
#https://stackoverflow.com/questions/65891168/keras-modelcheckpoint-not-saving-but-earlystopping-is-working-fine-with-the-same

class CustomCallback(tf.keras.callbacks.Callback):
    
    def __init__(self,x_val,y_val):
        super().__init__()
        self.x_val = x_val
        self.y_val = y_val
        pass
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': []}
        pass
    
        
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['accuracy'].append(logs.get('accuracy'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history['val_accuracy'].append(logs.get('val_accuracy'))
            
        y_pred=tf.reshape((self.model.predict(self.x_val)),(1,-1)).numpy()[0]
        y_pred[y_pred==-1]=0
        y_pred=y_pred.astype('int32')
        y_true=self.y_val.astype('int32')
        
        file_path="saved_models/weights_"+'epoch_'+str(epoch)+'.hdf5' 
        
        if epoch>2:
            if self.history['val_accuracy'][-1]>=0.90:
                self.model.save(file_path)
                if all([self.history['val_accuracy'][-1]<=self.history['val_accuracy'][-2],self.history['val_accuracy'][-1]<=self.history['val_accuracy'][-3],self.history['val_accuracy'][-2]<=self.history['val_accuracy'][-3]]):
                    print('val_accuracy not improving since the last 2 epochs, hence stopping training')
                    self.model.stop_training=True
        

In [10]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=5, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

paths='saved_models/final_models/f_model_2.hdf5'

checkpoint_save = tf.keras.callbacks.ModelCheckpoint(paths, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


In [ ]:
nadam = tf.keras.optimizers.legacy.Nadam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=1e-4)

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])

logs='logs/model_1'
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=logs)

model.fit(train_datagen.flow(x_train, y_train,batch_size=128),epochs=150, validation_data=(x_cv, y_cv), callbacks=[tensorboard,checkpoint_save,learning_rate_reduction,CustomCallback(x_cv, y_cv)], verbose=1)

Epoch 1/150
313/313 [==============================] - ETA: 0s - loss: 2.0412 - accuracy: 0.2729
Epoch 1: val_accuracy improved from -inf to 0.27430, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 159s 410ms/step - loss: 2.0412 - accuracy: 0.2729 - val_loss: 2.0479 - val_accuracy: 0.2743 - lr: 0.1000
Epoch 2/150
313/313 [==============================] - ETA: 0s - loss: 1.7306 - accuracy: 0.3692
Epoch 2: val_accuracy improved from 0.27430 to 0.31030, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 122s 391ms/step - loss: 1.7306 - accuracy: 0.3692 - val_loss: 2.0227 - val_accuracy: 0.3103 - lr: 0.1000
Epoch 3/150
313/313 [==============================] - ETA: 0s - loss: 1.5958 - accuracy: 0.4179
Epoch 3: val_accuracy did not improve from 0.31030
313/313 [==============================] - 122s 389ms/step - loss: 1.5958 - accuracy: 0.4179 - val_loss: 2.3367 - val_accuracy: 0.2949 - lr

Epoch 44/150
313/313 [==============================] - ETA: 0s - loss: 0.4085 - accuracy: 0.8588
Epoch 44: val_accuracy did not improve from 0.85320
313/313 [==============================] - 125s 401ms/step - loss: 0.4085 - accuracy: 0.8588 - val_loss: 0.4537 - val_accuracy: 0.8512 - lr: 0.0500
Epoch 45/150
313/313 [==============================] - ETA: 0s - loss: 0.4057 - accuracy: 0.8619
Epoch 45: val_accuracy did not improve from 0.85320

Epoch 45: ReduceLROnPlateau reducing learning rate to 0.02500000037252903.
313/313 [==============================] - 118s 375ms/step - loss: 0.4057 - accuracy: 0.8619 - val_loss: 0.7965 - val_accuracy: 0.7714 - lr: 0.0500
Epoch 46/150
313/313 [==============================] - ETA: 0s - loss: 0.3711 - accuracy: 0.8733
Epoch 46: val_accuracy improved from 0.85320 to 0.86190, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 125s 401ms/step - loss: 0.3711 - accuracy: 0.8733 - val_loss: 0.4291 - va

In [13]:
model=tf.keras.models.load_model('saved_models/final_models/f_model_2.hdf5')

In [ ]:
#Using pickle files to generate x_train & y_train, so that there is no data leakage
nadam = tf.keras.optimizers.legacy.Nadam(learning_rate=0.0125, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=1e-4)

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])

logs='logs/model_1'
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=logs)

model.fit(train_datagen.flow(x_train, y_train,batch_size=128),epochs=150, validation_data=(x_cv, y_cv), callbacks=[tensorboard,checkpoint_save,learning_rate_reduction,CustomCallback(x_cv, y_cv)], verbose=1)

Epoch 1/150
313/313 [==============================] - ETA: 0s - loss: 0.3304 - accuracy: 0.8870
Epoch 1: val_accuracy improved from -inf to 0.86640, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 160s 412ms/step - loss: 0.3304 - accuracy: 0.8870 - val_loss: 0.4409 - val_accuracy: 0.8664 - lr: 0.0125
Epoch 2/150
313/313 [==============================] - ETA: 0s - loss: 0.3263 - accuracy: 0.8884
Epoch 2: val_accuracy improved from 0.86640 to 0.87450, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 125s 398ms/step - loss: 0.3263 - accuracy: 0.8884 - val_loss: 0.3911 - val_accuracy: 0.8745 - lr: 0.0125
Epoch 3/150
313/313 [==============================] - ETA: 0s - loss: 0.3229 - accuracy: 0.8900
Epoch 3: val_accuracy improved from 0.87450 to 0.88020, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 124s 397ms/step - loss: 0.3229 - a

Epoch 23/150
313/313 [==============================] - ETA: 0s - loss: 0.2812 - accuracy: 0.9054
Epoch 23: val_accuracy did not improve from 0.88600
313/313 [==============================] - 126s 404ms/step - loss: 0.2812 - accuracy: 0.9054 - val_loss: 0.3791 - val_accuracy: 0.8823 - lr: 0.0016
Epoch 24/150
313/313 [==============================] - ETA: 0s - loss: 0.2822 - accuracy: 0.9046
Epoch 24: val_accuracy improved from 0.88600 to 0.88630, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 123s 393ms/step - loss: 0.2822 - accuracy: 0.9046 - val_loss: 0.3735 - val_accuracy: 0.8863 - lr: 0.0016
Epoch 25/150
 64/313 [=====>........................] - ETA: 1:30 - loss: 0.2762 - accuracy: 0.9059

In [9]:
model=tf.keras.models.load_model('saved_models/final_models/f_model_2.hdf5')

In [ ]:
#Using pickle files to generate x_train & y_train, so that there is no data leakage
nadam = tf.keras.optimizers.legacy.Nadam(learning_rate=0.0016, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=1e-4)

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])

logs='logs/model_1'
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=logs)

model.fit(train_datagen.flow(x_train, y_train,batch_size=128),epochs=50, validation_data=(x_cv, y_cv), callbacks=[tensorboard,checkpoint_save,learning_rate_reduction,CustomCallback(x_cv, y_cv)], verbose=1)

Epoch 1/50
313/313 [==============================] - ETA: 0s - loss: 0.2820 - accuracy: 0.9033
Epoch 1: val_accuracy improved from -inf to 0.88380, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 181s 471ms/step - loss: 0.2820 - accuracy: 0.9033 - val_loss: 0.3773 - val_accuracy: 0.8838 - lr: 0.0016
Epoch 2/50
313/313 [==============================] - ETA: 0s - loss: 0.2804 - accuracy: 0.9043
Epoch 2: val_accuracy did not improve from 0.88380
313/313 [==============================] - 129s 413ms/step - loss: 0.2804 - accuracy: 0.9043 - val_loss: 0.3825 - val_accuracy: 0.8813 - lr: 0.0016
Epoch 3/50
313/313 [==============================] - ETA: 0s - loss: 0.2762 - accuracy: 0.9043
Epoch 3: val_accuracy did not improve from 0.88380
313/313 [==============================] - 129s 413ms/step - loss: 0.2762 - accuracy: 0.9043 - val_loss: 0.3793 - val_accuracy: 0.8824 - lr: 0.0016
Epoch 4/50
313/313 [==============================] - ET

In [11]:
model=tf.keras.models.load_model('saved_models/final_models/f_model_2.hdf5')

In [ ]:
#Using pickle files to generate x_train & y_train, so that there is no data leakage
nadam = tf.keras.optimizers.legacy.Nadam(learning_rate=8.0000e-04, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=1e-4)

model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['accuracy'])

logs='logs/model_1'
tensorboard=tf.keras.callbacks.TensorBoard(log_dir=logs)

model.fit(train_datagen.flow(x_train, y_train,batch_size=128),epochs=50, validation_data=(x_cv, y_cv), callbacks=[tensorboard,checkpoint_save,learning_rate_reduction,CustomCallback(x_cv, y_cv)], verbose=1)

Epoch 1/50
313/313 [==============================] - ETA: 0s - loss: 0.2747 - accuracy: 0.9054
Epoch 1: val_accuracy improved from -inf to 0.88510, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 178s 424ms/step - loss: 0.2747 - accuracy: 0.9054 - val_loss: 0.3681 - val_accuracy: 0.8851 - lr: 8.0000e-04
Epoch 2/50
313/313 [==============================] - ETA: 0s - loss: 0.2758 - accuracy: 0.9054
Epoch 2: val_accuracy improved from 0.88510 to 0.88520, saving model to saved_models/final_models/f_model_2.hdf5
313/313 [==============================] - 124s 397ms/step - loss: 0.2758 - accuracy: 0.9054 - val_loss: 0.3690 - val_accuracy: 0.8852 - lr: 8.0000e-04
Epoch 3/50
313/313 [==============================] - ETA: 0s - loss: 0.2694 - accuracy: 0.9071
Epoch 3: val_accuracy did not improve from 0.88520
313/313 [==============================] - 125s 400ms/step - loss: 0.2694 - accuracy: 0.9071 - val_loss: 0.3697 - val_accuracy: 0.8851

Epoch 23/50
313/313 [==============================] - ETA: 0s - loss: 0.2670 - accuracy: 0.9099
Epoch 23: val_accuracy did not improve from 0.88670

Epoch 23: ReduceLROnPlateau reducing learning rate to 0.0001.
313/313 [==============================] - 126s 403ms/step - loss: 0.2670 - accuracy: 0.9099 - val_loss: 0.3691 - val_accuracy: 0.8855 - lr: 2.0000e-04
Epoch 24/50
313/313 [==============================] - ETA: 0s - loss: 0.2660 - accuracy: 0.9083
Epoch 24: val_accuracy did not improve from 0.88670
313/313 [==============================] - 123s 393ms/step - loss: 0.2660 - accuracy: 0.9083 - val_loss: 0.3696 - val_accuracy: 0.8846 - lr: 1.0000e-04
Epoch 25/50
313/313 [==============================] - ETA: 0s - loss: 0.2696 - accuracy: 0.9090
Epoch 25: val_accuracy did not improve from 0.88670
313/313 [==============================] - 124s 397ms/step - loss: 0.2696 - accuracy: 0.9090 - val_loss: 0.3699 - val_accuracy: 0.8860 - lr: 1.0000e-04
Epoch 26/50
313/313 [=============

In [9]:
model=tf.keras.models.load_model('saved_models/final_models/f_model_2.hdf5')

In [10]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 23s 39ms/step - loss: 0.3799 - accuracy: 0.8822


[0.3799421787261963, 0.8822000026702881]

In [4]:
%tensorboard --logdir logs/model_1